# Combined Parallel Model - Version 1

## Used only for sponsor demonstration

### Use version 2  instead (below) for more efficient (faster) and more complete code 

In [ ]:
# Imports
import detectron2
import contextlib
import datetime
import io
import os
import json
import logging
import cv2
import random
import numpy as np
import copy,torch,torchvision
import PIL
from PIL import Image
import xml.etree.ElementTree as X
import math
from itertools import repeat
import itertools
import re
import shutil
import io
import ast

from fvcore.common.file_io import PathManager
from fvcore.common.timer import Timer

from detectron2.structures import Boxes, BoxMode, PolygonMasks
from detectron2.config import *
from detectron2.modeling import build_model
from detectron2 import model_zoo
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import RotatedCOCOEvaluator,DatasetEvaluators, inference_on_dataset, coco_evaluation,DatasetEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

import matplotlib.pyplot as plt
from platform import python_version

import glob
import time
import shutil
from multiprocessing.pool import ThreadPool
import concurrent.futures
#from pyunpack import Archive

import torch
torch.cuda.set_device(0)

from torch.utils.cpp_extension import CUDA_HOME
print(torch.cuda.is_available(), CUDA_HOME)

print('\n')

setup_logger()

print('Python version:',python_version(),'\nOpencv version:',cv2.__version__,'\nTorch version:', torch.__version__, '\nTorch - Cuda is available:', torch.cuda.is_available(),'\nCudnn is enabled:', torch.backends.cudnn.enabled, '\nCuda version:', torch.version.cuda,'\nTorch_vision version:', torchvision.__version__)


"""
PointRend Training Script.
This script is a simplified version of the training script in detectron2/tools.
"""

import os

import detectron2.data.transforms as T
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    SemSegEvaluator,
    verify_results,
)
from detectron2.projects.point_rend import ColorAugSSDTransform, add_pointrend_config

In [65]:
# To prepare data
# To prepare data
def make_seg_cotton_dicts(Train_data_path, image_id = 1):
    import io
    import ast
    dataset_list = []
    subset_folders = os.listdir(Train_data_path)

    for frames in subset_folders:
        if '.png' in frames:
            dict_holder = {}
            file_name = os.path.join(Train_data_path, frames)
            dict_holder["file_name"] = file_name
            dict_holder["height"], dict_holder["width"] = cv2.imread(file_name).shape[0:2]
            dict_holder["image_id"] = image_id
            dict_holder["fr_name"] = re.sub(r'\.png','',frames)
            #s = open(file_name[0:-4] + '.txt').read().replace(':','')
            annotations = []
            with open(file_name[0:-4] + '.txt') as folder:
                for (k,line) in enumerate(folder):
                    tmp = line.split('[')
                    segment = [ast.literal_eval('['+tmp[1])] # format = [[float]]
                    cat_n_bbox = tmp[0].split()
                    category = int(cat_n_bbox[0].replace(':', ''))
                    bbox = [float(cat_n_bbox[1]), float(cat_n_bbox[2]), float(cat_n_bbox[3]), float(cat_n_bbox[4])]
                    # dict_store has boxmode(0) = [x1,y1,x2,y2] not boxmode(1) = [x1,y1,w,h] as previous code (use code cautiously)
                    dict_annot = {
                        "bbox": bbox,
                        "bbox_mode": detectron2.structures.BoxMode(0),
                        "category_id": category,
                        "segmentation": segment
                    }
                    annotations.append(dict_annot)

                    

            dict_holder["annotations"] = annotations
            #bboxes = np.loadtxt(io.StringIO(s), usecols=(4,))
            
            if 'train' or 'val' or 'test' in Train_data_path:
                dataset_list.append(dict_holder)
                image_id += 1
            # what about the augmented images --> it does not append augmented images with this code? (else is valid for validation and test data)
            else:
                if 'aug' not in frames:
                    dataset_list.append(dict_holder)
                    image_id += 1
    
    return dataset_list


def get_rbbox(mask):
    import cv2
    cnts, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rbbox = cv2.minAreaRect(cnts[0])
    return rbbox



def make_rbbox_cotton_dicts(Train_data_path, image_id = 1):

    padded_seg_dicts = make_seg_cotton_dicts(Train_data_path)

    dataset_list = []
    for file in padded_seg_dicts:

        img_height = file['height']
        img_width = file['width']
        img_path = file['file_name']
        frame_name = file['fr_name']

        dict_holder = {}
        dict_holder["file_name"] = img_path
        dict_holder["height"] =  img_height
        dict_holder["width"] = img_width
        dict_holder["image_id"] = image_id
        dict_holder["fr_name"] = frame_name

        # loop over each instance in current image and save annotations dictionary in a list
        annotations = []
        for index,variable in enumerate(file['annotations']):
            category = variable['category_id']
            segment = variable['segmentation']
            mymask = detectron2.structures.polygons_to_bitmask(segment, img_height,img_width)
            mymask = 255*mymask
            rbbox = get_rbbox((mymask).astype('uint8'))
            cent_x = rbbox[0][0]
            cent_y = rbbox[0][1]
            w = rbbox[1][0]
            h = rbbox[1][1]
            angle = rbbox[2]
#             if h > w:
#                 angle = 90-angle
#             else:
            angle = -angle # -angle works best (for now)
            bbox = [cent_x, cent_y, w, h, angle]
            bbox_mode = detectron2.structures.BoxMode(4) # box_mode = 4 --> (x_cent,y_cent,w,h,a)
            dict_annot = {
                            "bbox": bbox,
                            "bbox_mode": bbox_mode,
                            "category_id": category,
                        }
            annotations.append(dict_annot)

        dict_holder["annotations"] = annotations

        if 'train' or 'val' or 'test' in Train_data_path:
                    dataset_list.append(dict_holder)
                    image_id += 1
        else:
            if 'aug' in frame_name:
                dataset_list.append(dict_holder)
                image_id += 1
                
    return dataset_list


# Only works for 1 class (1 foreground and 1 background)
def make_sem_seg_cotton_dicts(Train_data_path, image_id = 1, complement = True):
    import io, cv2, re
    import numpy as np
    import ast
    import shutil
    import os
    import detectron2
    from detectron2 import structures
    
    dataset_list = []
    subset_folders = os.listdir(Train_data_path)
    save_path = os.path.join(Train_data_path,'Sem_Seg_GT_files')
    png_images = [name for name in os.listdir(Train_data_path) if '.png' in name]
    
    # check if path specified by the user has .png files
    if '.png' in png_images[0]:
        # if .png exists, continue to create the gt folder where semantic masks will be saved, else raise exception and stop execution of code
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        elif os.path.exists(save_path):
            shutil.rmtree(save_path , ignore_errors=True)
            os.makedirs(save_path)
    else:
        raise Exception("Sorry, no .png images were found in " + Train_data_path + ".\nPlease specify the correct path to training/test data")

    
    # Create the custom semantic segmentation dictionary
    for frames in subset_folders:
        if '.png' in frames:
            dict_holder = {}
            
            file_name = os.path.join(Train_data_path, frames)
            image_height, image_width = cv2.imread(file_name).shape[0:2]
            
            dict_holder["file_name"] = file_name
            dict_holder["height"], dict_holder["width"] = image_height, image_width
            dict_holder["image_id"] = image_id
            dict_holder["fr_name"] = re.sub(r'\.png','',frames)
            
            # Read instance labels and convert to unified mask for semantic segmentations tasks
            mask_accumulate = []
            with open(file_name[0:-4] + '.txt') as folder:
                for (k,line) in enumerate(folder):
                    tmp = line.split('[')
                    segment = [ast.literal_eval('['+tmp[1])] # format = [[float]]
                    
                    #cat_n_bbox = tmp[0].split()
                    #category = int(cat_n_bbox[0].replace(':', '')) # can still utilize category to get correct pixel values but, need to change the code (mask_sum*category)
                    
                    # Get current instance segmentation (polygon) & convert to mask
                    mask = detectron2.structures.polygons_to_bitmask(segment, image_height,image_width)
                    mask_accumulate.append(mask)
            
            # Unify instances to single semantic mask with pixel values indicating category
            mask_sum = sum(mask_accumulate)
            indices = np.where(mask_sum > 1)
            mask_sum[indices] = 1

            if complement == True:
                # Complement (foreground to bakcground) of unified mask of 0s and 1s
                image_height, image_width = cv2.imread(file_name).shape[0:2]
                full_image = np.ones((image_height, image_width))
                mask_sum = full_image - mask_sum # training targets are converted from foreground to background after complementing)

            # Save bit mask grayscale image of 1s and 0s to ground truth folder and to dictionary 
            sem_seg_file_name = os.path.join(save_path,frames)
            cv2.imwrite(sem_seg_file_name, mask_sum.astype(np.uint8))
            dict_holder["sem_seg_file_name"] = sem_seg_file_name



            if 'train' in Train_data_path:
                dataset_list.append(dict_holder)
                image_id += 1
            # what about the augmented images --> it does not append augmented images with this code? (else is valid for validation and test data)
            else:
                if 'aug' not in frames:
                    dataset_list.append(dict_holder)
                    image_id += 1
    
    return dataset_list

In [66]:
# Paths to data
base_path = r'E:\Fiber_Segmentation_codes\Data\Seg_files\New_Seg_Files_10_08_2022\Yildirim\Raw_10_08_2022'
train_path = os.path.join(base_path,'test_padded_v2')

In [67]:
for d in ["train_padded","test_padded_v2"]: #,,"val","test" (enter inside list for val data creation)
    DatasetCatalog.register("CFH_" + d + '_rotated',lambda d=d: make_rbbox_cotton_dicts(os.path.join(base_path,d)))
    MetadataCatalog.get("CFH_" + d + '_rotated').thing_classes=["fiber"]
    MetadataCatalog.get("CFH_" + d + '_rotated').thing_colors=[(0,0,255)]

In [68]:
for d in ["train_padded","test_padded_v2"]: #,,"val","test" (enter inside list for val data creation)
    DatasetCatalog.register("CFH_" + d + '_semantic',lambda d=d: make_sem_seg_cotton_dicts(os.path.join(base_path,d)))
    MetadataCatalog.get("CFH_" + d + '_semantic').stuff_classes=["fiber","background"]
    MetadataCatalog.get("CFH_" + d + '_semantic').stuff_colors=[(0,0,255),(0,255,0)]
    MetadataCatalog.get("CFH_" + d + '_semantic').evaluator_type = "sem_seg"

In [ ]:
DatasetCatalog.get("CFH_test_padded_v2_rotated")

In [ ]:
DatasetCatalog.get("CFH_test_padded_v2_semantic")

In [ ]:
DatasetCatalog.get("CFH_train_padded_rotated")

In [ ]:
DatasetCatalog.get("CFH_train_padded_semantic")

In [ ]:
"""
Rotated Faster RCNN configs
"""

cfg_RFRCNN = get_cfg()

cfg_RFRCNN.OUTPUT_DIR = r'E:\Fiber_Segmentation_codes\Yildirim\Training\Faster_RCNN\Rotated_Faster_RCNN_10_10_2022'

#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg_RFRCNN.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg_RFRCNN.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml") # Let training initialize from model zoo
cfg_RFRCNN.DATASETS.TRAIN = ("CFH_test_padded_v2_rotated",)
cfg_RFRCNN.DATASETS.TEST = ()

cfg_RFRCNN.MODEL.MASK_ON=False
cfg_RFRCNN.MODEL.PROPOSAL_GENERATOR.NAME = "RRPN"
cfg_RFRCNN.MODEL.RPN.HEAD_NAME = "StandardRPNHead"
cfg_RFRCNN.MODEL.RPN.BBOX_REG_WEIGHTS = (1.0,1.0,1.0,1.0,1.0)
cfg_RFRCNN.MODEL.ANCHOR_GENERATOR.NAME = "RotatedAnchorGenerator"
cfg_RFRCNN.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90,-60,-30,0,30,60,90]]
cfg_RFRCNN.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9 
cfg_RFRCNN.MODEL.ROI_HEADS.NAME = "RROIHeads"
cfg_RFRCNN.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   #this is far lower than usual.  
cfg_RFRCNN.MODEL.ROI_HEADS.NUM_CLASSES =1
cfg_RFRCNN.MODEL.ROI_BOX_HEAD.POOLER_TYPE = "ROIAlignRotated"
cfg_RFRCNN.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS = (1.0,1.0,1.0,1.0,1.0)
# cfg.MODEL.ROI_BOX_HEAD.NUM_CONV=4
# cfg.MODEL.ROI_MASK_HEAD.NUM_CONV=8
cfg_RFRCNN.SOLVER.IMS_PER_BATCH = 14 #can be up to  24 for a p100 (6 default)
cfg_RFRCNN.SOLVER.CHECKPOINT_PERIOD=1000
cfg_RFRCNN.SOLVER.BASE_LR = 0.005
# cfg.SOLVER.GAMMA=0.5
# cfg.SOLVER.STEPS=[1000,2000,4000,8000, 12000]
cfg_RFRCNN.SOLVER.MAX_ITER=5000


cfg_RFRCNN.DATALOADER.NUM_WORKERS = 1
cfg_RFRCNN.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True 
# cfg.DATALOADER.SAMPLER_TRAIN= "RepeatFactorTrainingSampler"
# cfg.DATALOADER.REPEAT_THRESHOLD=0.01
os.makedirs(cfg_RFRCNN.OUTPUT_DIR, exist_ok=True)#lets just check our output dir exists
# cfg.MODEL.BACKBONE.FREEZE_AT=6

In [ ]:
"""
Rotated Faster RCNN Predictor
"""
cfg_RFRCNN.MODEL.WEIGHTS = os.path.join(cfg_RFRCNN.OUTPUT_DIR, "model_final.pth")

# predictor = RotatedPredictor(cfg)
predictor_RFRCNN = DefaultPredictor(cfg_RFRCNN)

In [ ]:
"""
Pointrend Semantic Segmentation configs
"""

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_pointrend_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    #cfg.freeze()
    default_setup(cfg, args)
    return cfg

# Additional arguments for cfg
import argparse

args = argparse.Namespace()
args.config_file = r"C:\Users\ykocoglu\detectron2\projects\PointRend\configs\SemanticSegmentation\pointrend_semantic_R_101_FPN_1x_cityscapes.yaml "
args.opts = []
args.eval_only = "False"

cfg_PRSS = setup(args)

cfg_PRSS.DATASETS.TRAIN = ("CFH_test_padded_v2_padded",)
cfg_PRSS.DATASETS.TEST = () # () --> if no validation data,
cfg_PRSS.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 2
cfg_PRSS.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg_PRSS.MODEL.POINT_HEAD.NUM_CLASSES  = 2
cfg_PRSS.MODEL.ROI_BOX_HEAD.FED_LOSS_NUM_CLASSES = 2
cfg_PRSS.MODEL.RETINANET.NUM_CLASSES = 2

cfg_PRSS.MODEL.PANOPTIC_FPN.STUFF_AREA_LIMIT = 2896804
cfg_PRSS.MODEL.PANOPTIC_FPN.COMBINE.STUFF_AREA_LIMIT = 2896804

#cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE = 0 # use this if want to ignore fiber class pixels?
cfg_PRSS.SOLVER.IMS_PER_BATCH = 2
cfg_PRSS.SOLVER.STEPS = (16000,20000)
cfg_PRSS.SOLVER.MAX_ITER = 24000
cfg_PRSS.SOLVER.CHECKPOINT_PERIOD = 2000
cfg_PRSS.OUTPUT_DIR = r'E:\Fiber_Segmentation_codes\Yildirim\Training\Semantic_Seg\PointRend\Background_training'
cfg_PRSS.DATALOADER.NUM_WORKERS = 1 
cfg_PRSS.SOLVER.BASE_LR = 0.00025
cfg_PRSS.INPUT.MASK_FORMAT = "bitmask"
cfg_PRSS.MODEL.MASK_ON = True

# cfg.MODEL.SEM_SEG_HEAD.NORM = "BN" # A hacky way to disable syncbn to avoid errors in training due to request for multi gpu
# cfg.MODEL.RESNETS.NORM = "BN"
#cfg.MODEL.SEM_SEG_HEAD.PROJECT_CHANNELS = [64]
cfg_PRSS.INPUT.CROP.ENABLED = False
os.makedirs(cfg_PRSS.OUTPUT_DIR, exist_ok=True)

In [ ]:
"""
Pointrend Semantic Segmentation Predictor
"""
cfg_PRSS.MODEL.WEIGHTS = os.path.join(cfg_PRSS.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # set a custom testing threshold
predictor_PRSS = DefaultPredictor(cfg_PRSS)

In [ ]:
train_meta = MetadataCatalog.get("CFH_test_padded_v2_rotated").set(box_mode=detectron2.structures.BoxMode(4))
test_dict = DatasetCatalog.get("CFH_test_padded_v2_rotated")


test_image_path = r'E:\Fiber_Segmentation_codes\Data\Seg_files\New_Seg_Files_10_08_2022\Yildirim\Raw_10_08_2022\test_padded_v2'
test_images = [name for name in os.listdir(test_image_path) if '.png' in name]

combined_results_path = r'E:\Fiber_Segmentation_codes\Yildirim\Combined_Results\Combined_Results_Test'


for d in test_images:
    test_path = os.path.join(test_image_path,d)
    im = cv2.imread(test_path)
    #im = cv2.imread(test_image_path)

    outputs_RFRCNN = predictor_RFRCNN(im)
    myoutput = outputs_RFRCNN['instances'].pred_boxes.tensor.cpu().detach().numpy()

    outputs_PRSS = predictor_PRSS(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    myseg = torch.argmax(outputs_PRSS['sem_seg'],0).cpu().detach().numpy()

    newseg=(~myseg.astype(bool)).astype(int) # reverse binary digits for mask (change to mapping for more than 2 classes)
    newseg = (newseg*255).astype(np.uint8)
    newseg = cv2.cvtColor(newseg,cv2.COLOR_GRAY2RGB)

    for box in myoutput:
        new_box = ((box[0],box[1]),(box[2],box[3]),(-box[4]))
        new_box = cv2.boxPoints(new_box)
        new_box = np.int0(new_box)
        cv2.drawContours(newseg,[new_box],0,(0,0,255),3)

    cv2.imwrite(os.path.join(combined_results_path,d), newseg)


In [ ]:
myoutput = outputs_RFRCNN['instances'].pred_boxes.tensor.cpu().detach().numpy()

In [ ]:
myoutput

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

#center of rectangle
X = myoutput[0][0]
Y = myoutput[0][1]

center = np.array([[X],[Y]])

angle_deg = myoutput[0][4] #angle rectangle
angle = math.radians(angle_deg)

# rectangle's dimension
width = myoutput[0][2]
height = myoutput[0][3]

R_lt = np.array([[np.cos(angle),-np.sin(angle)],[-np.sin(angle),-np.cos(angle)]])
A = np.dot(R_lt,np.array([[width/2], [height/2]])) + center

R_rt = np.array([[np.cos(angle),np.sin(angle)],[-np.sin(angle),np.cos(angle)]])
B = np.dot(R_rt,np.array([[width/2], [height/2]])) + center

R_rb = np.array([[-np.cos(angle),np.sin(angle)],[np.sin(angle),np.cos(angle)]])
C = np.dot(R_rb,np.array([[width/2], [height/2]])) + center

R_lb = np.array([[-np.cos(angle),-np.sin(angle)],[np.sin(angle),-np.cos(angle)]])
D = np.dot(R_lb,np.array([[width/2], [height/2]])) + center

A = list(zip(*A))
B = list(zip(*B))
C = list(zip(*C))
D = list(zip(*D))

corners = [*A,*B,*C,*D]

In [ ]:
corners

In [ ]:
def is_on_right_side(x, y, xy0, xy1):
    x0, y0 = xy0
    x1, y1 = xy1
    a = float(y1 - y0)
    b = float(x0 - x1)
    c = - a*x0 - b*y0
    return a*x + b*y + c >= 0

def test_point(x, y, vertices):
    num_vert = len(vertices)
    is_right = [is_on_right_side(x, y, vertices[i], vertices[(i + 1) % num_vert]) for i in range(num_vert)]
    all_left = not any(is_right)
    all_right = all(is_right)
    return all_left or all_right

In [ ]:
indices = np.where(im[:,:,0] == im[:,:,0])
coordinates = list(zip(indices[0], indices[1])) #list(set(list(zip(indices[0], indices[1]))))

In [ ]:
# inner_points_x = []
# inner_points_y = []

# for index,point in enumerate(coordinates):
#     if not test_point(point[0],point[1],corners):
#         inner_points_x.append(point[0])
#         inner_points_y.append(point[1])
        
# newseg[(np.array(inner_points_y),np.array(inner_points_x))] = 0
# newseg = newseg*255



In [ ]:
def dot_prod_with_shared_start(start, end1, end2):

    return (end1[0] - start[0]) * (end2[0] - start[0]) + (end1[1] - start[1]) * (end2[1] - start[1])

def is_inside_rectangle(vertices, point):
    

    return all(dot_prod_with_shared_start(vertices[i - 1], v, point) > 0 for i, v in enumerate(vertices))

rect_vertices = corners

inner_points_x = []
inner_points_y = []
for index,point in enumerate(coordinates):
    condition = is_inside_rectangle(rect_vertices, point)
    if condition == False:
        inner_points_x.append(point[0])
        inner_points_y.append(point[1])

newseg[(np.array(inner_points_y),np.array(inner_points_x))] = 0
newseg = newseg*255        
        

In [ ]:
new_img = (newseg).astype(np.uint8)
imS = cv2.resize(new_img, (800, 800))
cv2.imshow('mask',imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
im_resize = cv2.resize(im, (800, 800))
cv2.imshow('mask',im_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
im_resize = cv2.resize((myseg*255).astype(np.uint8), (800, 800))
cv2.imshow('mask',im_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# For demo of longest path in skeleton (no other reason)

#train_meta = MetadataCatalog.get("CFH_test_padded_v2_rotated").set(box_mode=detectron2.structures.BoxMode(4))
test_dict_rotated = DatasetCatalog.get("CFH_test_padded_v2_rotated")
train_dict_rotated = DatasetCatalog.get("CFH_train_padded_rotated")

#test_dict_semantic = DatasetCatalog.get("CFH_test_padded_v2_semantic")
#train_dict_semantic = DatasetCatalog.get("CFH_train_padded_semantic")

#test_image_path = r'E:\Fiber_Segmentation_codes\Data\Seg_files\New_Seg_Files_10_08_2022\Yildirim\Raw_10_08_2022\test_padded_v2'
#test_images = [name for name in os.listdir(test_image_path) if '.png' in name]

original_image_path = r'C:\Users\ykocoglu\Desktop\Skeleton_longest_path_images'
original_images = [name for name in os.listdir(original_image_path) if '.png' in name]

#combined_results_path = r'E:\Fiber_Segmentation_codes\Yildirim\Combined_Results\Combined_Results_Test'


# for d in test_images:
#     test_path = os.path.join(test_image_path,d)
#     im = cv2.imread(test_path)
#     #im = cv2.imread(test_image_path)

#     outputs_RFRCNN = predictor_RFRCNN(im)
#     myoutput = outputs_RFRCNN['instances'].pred_boxes.tensor.cpu().detach().numpy()

#     outputs_PRSS = predictor_PRSS(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
#     myseg = torch.argmax(outputs_PRSS['sem_seg'],0).cpu().detach().numpy()

#     newseg=(~myseg.astype(bool)).astype(int) # reverse binary digits for mask (change to mapping for more than 2 classes)
#     newseg = (newseg*255).astype(np.uint8)
#     newseg = cv2.cvtColor(newseg,cv2.COLOR_GRAY2RGB)

#     for box in myoutput:
#         new_box = ((box[0],box[1]),(box[2],box[3]),(-box[4]))
#         new_box = cv2.boxPoints(new_box)
#         new_box = np.int0(new_box)
#         cv2.drawContours(newseg,[new_box],0,(0,0,255),3)

#     cv2.imwrite(os.path.join(combined_results_path,d), newseg)

In [ ]:
my_original_images = []
for index in range(0,len(original_images)):
    my_original_images.append(original_images[index][:-4])

In [ ]:
# train and test semantic GT images
semantic_path = r'C:\Users\ykocoglu\Desktop\Skeleton_longest_path_images\Binary images'
semantic_images = [name for name in os.listdir(semantic_path ) if '.png' in name]

In [ ]:
new_list_train = []
for index in range(0,len(train_dict_rotated)):
    if train_dict_rotated[index]['fr_name'] in my_original_images:
        new_list_train.append(train_dict_rotated[index])

In [ ]:
new_list_test = []
for index in range(0,len(test_dict_rotated)):
    if test_dict_rotated[index]['fr_name'] in my_original_images:
        new_list_test.append(test_dict_rotated[index])

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

kernel = np.ones((10, 10), np.uint8)

for index in range(0,len(semantic_images)): # len(semantic_images)
    current_frame = cv2.imread(os.path.join(semantic_path,semantic_images[index]), cv2.IMREAD_UNCHANGED)
    seg_img = cv2.bitwise_not(current_frame)
    seg_img = (254 - seg_img)/255
    new_seg_img = seg_img.astype(np.uint8)
    dilation = cv2.dilate(new_seg_img, kernel, iterations=1)
    imS = cv2.resize(dilation, (1000, 1000))
    cv2.imshow('image', imS*255)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
Per_fiber_binary_images_path = r'C:\Users\ykocoglu\Desktop\Skeleton_longest_path_images\Binary images\Per_Fiber_Binary_images'

In [ ]:
# Get binary individual fiber images (for sponsor demo)

import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

def is_on_right_side(x, y, xy0, xy1):
    x0, y0 = xy0
    x1, y1 = xy1
    a = float(y1 - y0)
    b = float(x0 - x1)
    c = - a*x0 - b*y0
    return a*x + b*y + c >= 0

def test_point(x, y, vertices):
    num_vert = len(vertices)
    is_right = [is_on_right_side(x, y, vertices[i], vertices[(i + 1) % num_vert]) for i in range(num_vert)]
    all_left = not any(is_right)
    all_right = all(is_right)
    return all_left or all_right

def dot_prod_with_shared_start(start, end1, end2):
    return (end1[0] - start[0]) * (end2[0] - start[0]) + (end1[1] - start[1]) * (end2[1] - start[1])

def is_inside_rectangle(vertices, point):
    return all(dot_prod_with_shared_start(vertices[i - 1], v, point) > 0 for i, v in enumerate(vertices))


import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

kernel = np.ones((10, 10), np.uint8)

for index in range(0,len(semantic_images)): # len(semantic_images)
    current_frame = cv2.imread(os.path.join(semantic_path,semantic_images[index]), cv2.IMREAD_UNCHANGED)
    seg_img = cv2.bitwise_not(current_frame)
    seg_img = (254 - seg_img)/255
    new_seg_img = seg_img.astype(np.uint8)
    dilation = cv2.dilate(new_seg_img, kernel, iterations=1)
    
    print(semantic_images[index])

    for frame in new_list_train:
        if semantic_images[index][:-4] == frame['fr_name']:
            for j,bbox in enumerate(frame['annotations']):
                X = bbox['bbox'][0]
                Y = bbox['bbox'][1]
                center = np.array([[X],[Y]])
                angle_deg = bbox['bbox'][4] #angle rectangle
                angle = math.radians(angle_deg)

                # rectangle's dimension
                width = bbox['bbox'][2]
                height = bbox['bbox'][3]

                R_lt = np.array([[np.cos(angle),-np.sin(angle)],[-np.sin(angle),-np.cos(angle)]])
                A = np.dot(R_lt,np.array([[width/2], [height/2]])) + center

                R_rt = np.array([[np.cos(angle),np.sin(angle)],[-np.sin(angle),np.cos(angle)]])
                B = np.dot(R_rt,np.array([[width/2], [height/2]])) + center

                R_rb = np.array([[-np.cos(angle),np.sin(angle)],[np.sin(angle),np.cos(angle)]])
                C = np.dot(R_rb,np.array([[width/2], [height/2]])) + center

                R_lb = np.array([[-np.cos(angle),-np.sin(angle)],[np.sin(angle),-np.cos(angle)]])
                D = np.dot(R_lb,np.array([[width/2], [height/2]])) + center

                A = list(zip(*A))
                B = list(zip(*B))
                C = list(zip(*C))
                D = list(zip(*D))

                corners = [*A,*B,*C,*D]
                print(corners)
                print('\n')
                
#                 rect_vertices = corners

#                 indices = np.where(current_frame[:,:] == current_frame[:,:])
#                 coordinates = list(zip(indices[0], indices[1]))

#                 inner_points_x = []
#                 inner_points_y = []
#                 for i,point in enumerate(coordinates):
#                     condition = is_inside_rectangle(rect_vertices, point)
#                     if condition == False:
#                         inner_points_x.append(point[0])
#                         inner_points_y.append(point[1])
                
#                 # To keep the same segmentation image as reference and to avoid accumulating 0s in the same image for different bboxes (in for loop) 
#                 new_dilation = dilation # keep original seg image as reference for all bboxes (does not work and changes original dilation image too)
#                 new_dilation[(np.array(inner_points_y),np.array(inner_points_x))] = 0
#                 cv2.imwrite(os.path.join(Per_fiber_binary_images_path,frame['fr_name'] + '_' + str(j) +'.png'), new_dilation*255)
# #                 imS = cv2.resize(new_dilation, (1000, 1000))
# #                 imC = cv2.resize(dilation, (1000, 1000))
# #                 cv2.imshow('image', imC*255)
# #                 cv2.waitKey(0)
# #                 cv2.destroyAllWindows()
                
#                 # Need to bring back the original image for reference each time (it will accumulate 0s without this piece of code - do not delete this!)
#                 seg_img = cv2.bitwise_not(current_frame)
#                 seg_img = (254 - seg_img)/255
#                 new_seg_img = seg_img.astype(np.uint8)
#                 dilation = cv2.dilate(new_seg_img, kernel, iterations=1)
#         else:
#             print('Not this image')
            
#     for frame in new_list_test:
#         if semantic_images[index][:-4] == frame['fr_name']:
#             for j,bbox in enumerate(frame['annotations']):
#                 X = bbox['bbox'][0]
#                 Y = bbox['bbox'][1]
#                 center = np.array([[X],[Y]])
#                 angle_deg = bbox['bbox'][4] #angle rectangle
#                 angle = math.radians(angle_deg)

#                 # rectangle's dimension
#                 width = bbox['bbox'][2]
#                 height = bbox['bbox'][3]

#                 R_lt = np.array([[np.cos(angle),-np.sin(angle)],[-np.sin(angle),-np.cos(angle)]])
#                 A = np.dot(R_lt,np.array([[width/2], [height/2]])) + center

#                 R_rt = np.array([[np.cos(angle),np.sin(angle)],[-np.sin(angle),np.cos(angle)]])
#                 B = np.dot(R_rt,np.array([[width/2], [height/2]])) + center

#                 R_rb = np.array([[-np.cos(angle),np.sin(angle)],[np.sin(angle),np.cos(angle)]])
#                 C = np.dot(R_rb,np.array([[width/2], [height/2]])) + center

#                 R_lb = np.array([[-np.cos(angle),-np.sin(angle)],[np.sin(angle),-np.cos(angle)]])
#                 D = np.dot(R_lb,np.array([[width/2], [height/2]])) + center

#                 A = list(zip(*A))
#                 B = list(zip(*B))
#                 C = list(zip(*C))
#                 D = list(zip(*D))

#                 corners = [*A,*B,*C,*D]
                
#                 rect_vertices = corners

#                 indices = np.where(current_frame[:,:] == current_frame[:,:])
#                 coordinates = list(zip(indices[0], indices[1]))

#                 inner_points_x = []
#                 inner_points_y = []
#                 for i,point in enumerate(coordinates):
#                     condition = is_inside_rectangle(rect_vertices, point)
#                     if condition == False:
#                         inner_points_x.append(point[0])
#                         inner_points_y.append(point[1])
                
#                 # To keep the same segmentation image as reference and to avoid accumulating 0s in the same image for different bboxes (in for loop) 
#                 new_dilation = dilation # keep original seg image as reference for all bboxes (does not work and changes original dilation image too)
#                 new_dilation[(np.array(inner_points_y),np.array(inner_points_x))] = 0
#                 cv2.imwrite(os.path.join(Per_fiber_binary_images_path,frame['fr_name'] + '_' + str(j) +'.png'), new_dilation*255)
# #                 imS = cv2.resize(new_dilation, (1000, 1000))
# #                 imC = cv2.resize(dilation, (1000, 1000))
# #                 cv2.imshow('image', imC*255)
# #                 cv2.waitKey(0)
# #                 cv2.destroyAllWindows()
                
#                 # Need to bring back the original image for reference each time (it will accumulate 0s without this piece of code - do not delete this!)
#                 seg_img = cv2.bitwise_not(current_frame)
#                 seg_img = (254 - seg_img)/255
#                 new_seg_img = seg_img.astype(np.uint8)
#                 dilation = cv2.dilate(new_seg_img, kernel, iterations=1)
#         else:
#             print('Not this image')




In [ ]:
import cv2, os
import numpy as np

kernel = np.ones((10, 10), np.uint8)

for index in range(0,len(semantic_images)): # len(semantic_images)
    
    # current frame dilated
    current_frame = cv2.imread(os.path.join(semantic_path,semantic_images[index]), cv2.IMREAD_UNCHANGED)
    seg_img = cv2.bitwise_not(current_frame)
    seg_img = (254 - seg_img)/255
    new_seg_img = seg_img.astype(np.uint8)
    dilation = cv2.dilate(new_seg_img, kernel, iterations=1)
    
    print(semantic_images[index])
    
    # for training images 
    for frame in new_list_train:
        # if training image = current semantic image
        if semantic_images[index][:-4] == frame['fr_name']:
            # for each bbox
            for j,bbox in enumerate(frame['annotations']):
                rect = ((bbox['bbox'][0],bbox['bbox'][1]),(bbox['bbox'][2],bbox['bbox'][3]),-bbox['bbox'][4])
                pts = np.array(cv2.boxPoints(rect),np.int32) # corner point coordinates
                #print(cv2.boxPoints(rect))
                pts = pts.reshape((-1,1,2)) # x,y polygon points
                #pts = np.flip(pts)
                print(pts)
                print('\n')
                cv2.polylines(current_frame, [pts], False,(0,255,255))
                mask = np.zeros(current_frame.shape[:2],np.uint8)
                cv2.fillPoly(mask,[pts],(255,255,255))
                new_frame = cv2.bitwise_and(dilation,dilation,mask = mask)
                cv2.imshow("masked frame", new_frame*255)
                cv2.waitKey(0)
                cv2.destroyAllWindows()



# Combined Parallel Model - Version 2

## Use this! (more efficient and complete code)

In [1]:
# Full model (Rotated boxes + Semantic Seg + Longest Path Tracking)

# Required Libraries
import detectron2
import contextlib
import datetime
import io
import os
import json
import logging
import cv2
import random
import numpy as np
import copy,torch,torchvision
import PIL
from PIL import Image
import xml.etree.ElementTree as X
import math
from itertools import repeat
import itertools
import re
import shutil
import io
import ast

# Longest path libraries
import networkx as nx
import rdp
from skimage.morphology import skeletonize, medial_axis, thin
from skimage.util import invert
import sknw

from fvcore.common.file_io import PathManager
from fvcore.common.timer import Timer

from detectron2.structures import Boxes, BoxMode, PolygonMasks
from detectron2.config import *
from detectron2.modeling import build_model
from detectron2 import model_zoo
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import RotatedCOCOEvaluator,DatasetEvaluators, inference_on_dataset, coco_evaluation,DatasetEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

import matplotlib.pyplot as plt
from platform import python_version

import glob
import time
import shutil
from multiprocessing.pool import ThreadPool
import concurrent.futures
#from pyunpack import Archive

import torch
torch.cuda.set_device(0)

from torch.utils.cpp_extension import CUDA_HOME
print(torch.cuda.is_available(), CUDA_HOME)

print('\n')

setup_logger()

print('Python version:',python_version(),'\nOpencv version:',cv2.__version__,'\nTorch version:', torch.__version__, '\nTorch - Cuda is available:', torch.cuda.is_available(),'\nCudnn is enabled:', torch.backends.cudnn.enabled, '\nCuda version:', torch.version.cuda,'\nTorch_vision version:', torchvision.__version__)


"""
PointRend Training Script.
This script is a simplified version of the training script in detectron2/tools.
"""

import os

import detectron2.data.transforms as T
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    SemSegEvaluator,
    verify_results,
)
from detectron2.projects.point_rend import ColorAugSSDTransform, add_pointrend_config

C:\Users\ykocoglu\Anaconda3\envs\detectron_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3


Python version: 3.8.13 
Opencv version: 4.6.0 
Torch version: 1.10.0 
Torch - Cuda is available: True 
Cudnn is enabled: True 
Cuda version: 11.3 
Torch_vision version: 0.11.0


In [101]:
# Functions for finding max change of angle along the longest path (thresholds: tolerance, min_angle)
def angle(directions):
    """Return the angle between vectors
    """
    vec2 = directions[1:]
    vec1 = directions[:-1]

    norm1 = np.sqrt((vec1 ** 2).sum(axis=1))
    norm2 = np.sqrt((vec2 ** 2).sum(axis=1))
    cos = (vec1 * vec2).sum(axis=1) / (norm1 * norm2)
    
    return np.arccos(cos)

# Finds turning points with significant change in angle within simplified curve of longest path (tolerance = sample frequency of points, min_angle = threshold for significant change in angle)
# Currently simplified curve is calculated using frequency of 10 points and min_angle = 80 degrees anything greater than 80 degress will cause the overall algorithm to avoid the longest path found
def find_turning_points(longest_path,tolerance = 10,min_angle = np.pi/2.25):
    # Initialize points from longest path
    points = longest_path
    x, y = points.T

    # Use the Ramer-Douglas-Peucker algorithm to simplify the path
    # http://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm
    # Python implementation: https://github.com/sebleier/RDP/
    simplified = np.array(rdp.rdp(points.tolist(), tolerance))
    sx, sy = simplified.T

    # compute the direction vectors on the simplified curve
    directions = np.diff(simplified, axis=0)
    theta = angle(directions)
    
    # Select the index of the points with the greatest theta
    # Large theta is associated with greatest change in direction.
    idx = np.where(theta>min_angle)[0]+1
    
    return idx

# sliding window for node pairs
def sliding_window(l, window_size=2):
    if window_size > len(l):
        raise ValueError("Window size must be smaller or equal to the number of elements in the list.")

    t = []
    for i in range(0, window_size):
        t.append(l[i:])

    return zip(*t)

# Input: node
# Output: len(total non-zero pixels)
def get_end_point_nodes(graph,skel):
    
    # Find the largets connected network
    xxx = list(nx.connected_components(graph))
    largest_network_index = [len(item) for index,item in enumerate(xxx)].index(max([len(item) for index,item in enumerate(xxx)]))
    my_network = xxx[largest_network_index] 
    
    # Get endpoint nodes of the largest connected network
    endpoint_nodes = []
    for i in  my_network:#range(0,len(graph.nodes())):
        node = graph.nodes()[i]['o'].astype(int)
        x1       = node[0]
        y1       = node[1]
        x1_minus = node[0] - 1
        x1_plus  = node[0] + 1
        y1_minus = node[1] - 1
        y1_plus  = node[1] + 1

        mylist = []
        for j in [x1_minus,x1,x1_plus]:
            for k in [y1_minus,y1, y1_plus]:
                mylist.append(skel[j,k])

        nonzero_counts = np.count_nonzero(mylist)

        # If there are only 2 neighboring pixels that are non-zero then that node is an end-point node
        if nonzero_counts == 2:
            endpoint_nodes.append(i)
    
    return endpoint_nodes

# Find longest path of the fiber in a predicted instance 
# Only works with binary images
# tolerance: number of sampling points to smooth the path of the skeleton
# min_angle: threshold on angle that is considered significant
def longest_shortest_path(image,tolerance = 20,min_angle = np.pi/3, turn_on_rdp = False):

    # Get skeleton of given binary image
    skel = medial_axis(image).astype(np.uint8)

    # build graph from the skeleton
    graph = sknw.build_sknw(skel)

    # create_node_pairs from endpoint nodes:
    end_point_nodes = get_end_point_nodes(graph,skel)
    new_pts = np.array([graph.nodes[i]['o'] for i in end_point_nodes])
    all_pts = np.array([graph.nodes[i]['o'] for i in graph.nodes])
    paired_end_points = list(itertools.combinations(end_point_nodes,2))
    
    if len(paired_end_points) == 0:
        longest_path = [0]

    else:
        # Find longest path
        all_paths = []
        for s,e in paired_end_points:
            shortest_path = nx.shortest_path(graph,s,e)
            edge_pairs = list(sliding_window(shortest_path,2))
            sorted_edge_pairs = [tuple(sorted(item)) for item in edge_pairs]


            edge_path = []
            for ss,ee in sorted_edge_pairs:
                edge_segment = [x for x in graph[ss][ee]['pts']]
                edge_path.extend(edge_segment)

            all_paths.append(edge_path)

        longest_path_index = [len(x) for x in all_paths].index(max([len(x) for x in all_paths]))
        longest_path = np.array(all_paths[longest_path_index])

        # If rdp algorithm is turned on, use the rdp algorithm to find the longest path (By default, it's turned off)
        if turn_on_rdp:

            # check for turning points with significant angles (based on given tolerance and mind_angle threshold)
            # If turning points exist find another path where no significant turning points exist
            idx = find_turning_points(longest_path,tolerance = 20,min_angle = np.pi/3)

            while len(idx) != 0:
                all_paths.pop(longest_path_index)
                longest_path_index = [len(x) for x in all_paths].index(max([len(x) for x in all_paths]))
                longest_path = np.array(all_paths[longest_path_index])
                idx = find_turning_points(longest_path,tolerance = 20,min_angle = np.pi/3)
            
    return longest_path




In [3]:
"""
Rotated Faster RCNN Predictor
"""
# Config setup
cfg_RFRCNN = get_cfg()
cfg_RFRCNN.OUTPUT_DIR = r'E:\Fiber_Segmentation_codes\Yildirim\Training\Faster_RCNN\Rotated_Faster_RCNN_10_10_2022'

cfg_RFRCNN.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg_RFRCNN.MODEL.WEIGHTS = os.path.join(cfg_RFRCNN.OUTPUT_DIR, "model_final.pth")
cfg_RFRCNN.MODEL.PROPOSAL_GENERATOR.NAME = "RRPN"
cfg_RFRCNN.MODEL.ROI_HEADS.NAME = "RROIHeads"
cfg_RFRCNN.MODEL.ROI_HEADS.NUM_CLASSES =1
cfg_RFRCNN.MODEL.ROI_BOX_HEAD.POOLER_TYPE = "ROIAlignRotated"
cfg_RFRCNN.MODEL.ANCHOR_GENERATOR.NAME = "RotatedAnchorGenerator"
cfg_RFRCNN.MODEL.RPN.HEAD_NAME = "StandardRPNHead"
cfg_RFRCNN.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS = (1.0,1.0,1.0,1.0,1.0)
cfg_RFRCNN.MODEL.RPN.BBOX_REG_WEIGHTS = (1.0,1.0,1.0,1.0,1.0)
cfg_RFRCNN.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90,-60,-30,0,30,60,90]]

# Test thresholds
cfg_RFRCNN.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9 
cfg_RFRCNN.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.1

# predictor = RotatedPredictor(cfg)
predictor_RFRCNN = DefaultPredictor(cfg_RFRCNN)

[12/08 17:53:17 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [4]:
"""
Pointrend Semantic Segmentation configs
"""

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_pointrend_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    #cfg.freeze()
    default_setup(cfg, args)
    return cfg

# Additional arguments for cfg
import argparse

args = argparse.Namespace()
args.config_file = r"C:\Users\ykocoglu\detectron2\projects\PointRend\configs\SemanticSegmentation\pointrend_semantic_R_101_FPN_1x_cityscapes.yaml "
args.opts = []
args.eval_only = "False"

cfg_PRSS = setup(args)

cfg_PRSS.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 2
cfg_PRSS.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg_PRSS.MODEL.POINT_HEAD.NUM_CLASSES  = 2
cfg_PRSS.MODEL.ROI_BOX_HEAD.FED_LOSS_NUM_CLASSES = 2
cfg_PRSS.MODEL.RETINANET.NUM_CLASSES = 2

cfg_PRSS.MODEL.PANOPTIC_FPN.STUFF_AREA_LIMIT = 2896804
cfg_PRSS.MODEL.PANOPTIC_FPN.COMBINE.STUFF_AREA_LIMIT = 2896804

cfg_PRSS.OUTPUT_DIR = r'E:\Fiber_Segmentation_codes\Yildirim\Training\Semantic_Seg\PointRend\Background_training'
cfg_PRSS.INPUT.MASK_FORMAT = "bitmask"
cfg_PRSS.MODEL.MASK_ON = True
cfg_PRSS.INPUT.CROP.ENABLED = False

[12/08 17:53:17 detectron2]: Rank of current process: 0. World size: 1
[12/08 17:53:17 detectron2]: Rank of current process: 0. World size: 1
[12/08 17:53:17 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------------------------------------------
sys.platform            win32
Python                  3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]
numpy                   1.22.3
detectron2              0.6 @c:\users\ykocoglu\detectron2\detectron2
Compiler                MSVC 192227905
CUDA compiler           CUDA 11.3
detectron2 arch flags   c:\users\ykocoglu\detectron2\detectron2\_C.cp38-win_amd64.pyd; cannot find cuobjdump
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0 @C:\Users\ykocoglu\Anaconda3\envs\detectron_env\lib\site-packages\torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   NVIDIA TITAN RTX (arch=7.5)
Driver version          516.9

[12/08 17:53:17 detectron2]: Contents of args.config_file=C:\Users\ykocoglu\detectron2\projects\PointRend\configs\SemanticSegmentation\pointrend_semantic_R_101_FPN_1x_cityscapes.yaml :
_BASE_: Base-PointRend-Semantic-FPN.yaml
MODEL:
  WEIGHTS: detectron2://ImageNetPretrained/MSRA/R-101.pkl
  RESNETS:
    DEPTH: 101
  SEM_SEG_HEAD:
    NUM_CLASSES: 19
  POINT_HEAD:
    NUM_CLASSES: 19
    TRAIN_NUM_POINTS: 2048
    SUBDIVISION_NUM_POINTS: 8192
DATASETS:
  TRAIN: ("cityscapes_fine_sem_seg_train",)
  TEST: ("cityscapes_fine_sem_seg_val",)
SOLVER:
  BASE_LR: 0.01
  STEPS: (40000, 55000)
  MAX_ITER: 65000
  IMS_PER_BATCH: 32
INPUT:
  MIN_SIZE_TRAIN: (512, 768, 1024, 1280, 1536, 1792, 2048)
  MIN_SIZE_TRAIN_SAMPLING: "choice"
  MIN_SIZE_TEST: 1024
  MAX_SIZE_TRAIN: 4096
  MAX_SIZE_TEST: 2048
  CROP:
    ENABLED: True
    TYPE: "absolute"
    SIZE: (512, 1024)
    SINGLE_CATEGORY_MAX_AREA: 0.75
  COLOR_AUG_SSD: True
DATALOADER:
  NUM_WORKERS: 10

[12/08 17:53:17 detectron2]: Running with full

[12/08 17:53:17 detectron2]: Running with full config:
CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WORKERS: 10
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - cityscapes_fine_sem_seg_val
  TRAIN:
  - cityscapes_fine_sem_seg_train
GLOBAL:
  HACK: 1.0
INPUT:
  COLOR_AUG_SSD: true
  CROP:
    ENABLED: true
    SINGLE_CATEGORY_MAX_AREA: 0.75
    SIZE:
    - 512
    - 1024
    TYPE: absolute
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 2048
  MAX_SIZE_TRAIN: 4096
  MIN_SIZE_TEST: 1024
  MIN_SIZE_TRAIN:
  - 512
  - 768
  - 1024
  - 1280
  - 1536
  - 1792
  - 2048
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2.0
    NAME: Default

[12/08 17:53:17 detectron2]: Full config saved to ./output\config.yaml
[12/08 17:53:17 detectron2]: Full config saved to ./output\config.yaml
[12/08 17:53:18 d2.utils.env]: Using a generated random seed 18120905
[12/08 17:53:18 d2.utils.env]: Using a generated random seed 18120905


In [5]:
"""
Pointrend Semantic Segmentation Predictor
"""
cfg_PRSS.MODEL.WEIGHTS = os.path.join(cfg_PRSS.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # set a custom testing threshold
predictor_PRSS = DefaultPredictor(cfg_PRSS)

[12/08 17:53:18 fvcore.common.checkpoint]: [Checkpointer] Loading from E:\Fiber_Segmentation_codes\Yildirim\Training\Semantic_Seg\PointRend\Background_training\model_final.pth ...
[12/08 17:53:19 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                               | Names in Checkpoint                                                                                  | Shapes                                          |
|:---------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*            | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*            | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,

[12/08 17:53:19 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                               | Names in Checkpoint                                                                                  | Shapes                                          |
|:---------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*            | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*            | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*            | backbone.bottom_up.res2.0.conv3.{norm.bias,norm.running_

In [102]:
# Test images path
test_image_path = r'E:\Fiber_Segmentation_codes\Data\Seg_files\New_Seg_Files_10_08_2022\Yildirim\Raw_10_08_2022\test_padded_v2'
test_images = [name for name in os.listdir(test_image_path) if '.png' in name]

# Save results to given path below
combined_results_path = r'E:\Fiber_Segmentation_codes\Yildirim\Combined_Results\Full_pipeline_skeletons\Test_images'

# Video Path
video_path = r'E:\Fiber_Segmentation_codes\Data\Videos\all_videos'

# Control kernel size for morphological operations
kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(60,60)) #np.ones((10, 10), np.uint8)
kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(30,30))
kernel_dilate = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))

# Get combined output and save to given path (combined_results_path)
for index, d in enumerate(test_images):
    if index in list(range(0,len(test_images))):
        
        # Current image 
        test_path = os.path.join(test_image_path,d)
        im = cv2.imread(test_path)
        
        # Video of current image
        vid_name, frame_name = d.split('_')
        video = os.path.join(video_path,vid_name + '.avi')
        
        # Average frame from video
        capture = cv2.VideoCapture(video)
        totalframecount= int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_list = np.arange(0,totalframecount)
        
        cumulative_frames = []
        for frame_no in frame_list:
            capture.set(1,frame_no);
            success, frame = capture.read()
            cumulative_frames.append(frame)
    
        average_frame = np.mean(cumulative_frames, axis=0).astype(dtype=np.uint8) # Average frame
        
        # ***Pad average_frame image***
        pixels = 700
        
        old_image_height, old_image_width, channels = average_frame.shape
        
        new_image_width = old_image_width + pixels
        new_image_height = old_image_height + pixels
        color = (128,128,128)
        
        padded_average_frame = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)
        # compute center offset
        x_center = (new_image_width - old_image_width) // 2
        y_center = (new_image_height - old_image_height) // 2

        # copy img image into center of result image
        padded_average_frame[y_center:y_center+old_image_height, 
               x_center:x_center+old_image_width] = average_frame
        
        # ***Rotated Faster RCNN Output***
        outputs_RFRCNN = predictor_RFRCNN(im)
        myoutput = outputs_RFRCNN['instances'].pred_boxes.tensor.cpu().detach().numpy()
        
        # ***Semantic Segmentation Output for average frame***
        outputs_PRSS = predictor_PRSS(padded_average_frame)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
        myseg = torch.argmax(outputs_PRSS['sem_seg'],0).cpu().detach().numpy()
        newseg=(~myseg.astype(bool)).astype(np.uint8)# reverse binary digits for mask (change to mapping for more than 2 classes)
        
        
        # ***Morphological Operations on Semantic Segmentation Output***
        # Dilate - morphological operation (to thicken fiber segmentations slightly)
        dilation = cv2.dilate(newseg,kernel_dilate,iterations = 1)
        # Closing - morphological operation (to close connections - if any are open)
        closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_close)
        # Opening -  morphological operation (to remove noise)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel_open)
        #opening = cv2.resize(opening, (1000, 1000))
        #dilation = cv2.resize(dilation, (1000, 1000))
        #newseg = cv2.resize(newseg, (1000, 1000))
        
        
        # ***Skeletonization Per Bounding Box*** (Individualization of fibers)
        longest_path_curves = []
        for box in myoutput:
            rect = ((box[0],box[1]),(box[2],box[3]),(-box[4]))
            pts = np.array(cv2.boxPoints(rect),np.int32)
            pts = pts.reshape((-1,1,2))

            cv2.polylines(opening, [pts], False,(0,255,255))
            mask = np.zeros(opening.shape[:2],np.uint8)
            cv2.fillPoly(mask,[pts],(255,255,255))
            new_frame = cv2.bitwise_and(opening,opening,mask = mask)
        
            
            # Ignore bounding boxes with no fiber segments
            if 1 not in np.unique(new_frame):
                print('This bounding box has no fiber in it')
            else:
            # Skeletonize by finding longest shortest path
                longest_path = longest_shortest_path(new_frame,tolerance = 20,min_angle = np.pi/3, turn_on_rdp = False)
                if len(longest_path) < 2:
                    print('Not enough endpoints found')
                else:
                    reverse_yx = np.array([np.array([y,x]) for x,y in longest_path]) # reverse x,y to y,x - done to correctly visualize in cv2.imshow() 
                    reshaped_longest_path = reverse_yx.reshape((-1,1,2))
                    longest_path_curves.append(reshaped_longest_path)
            
        
        # ***Draw Detected Skeletons per cotton fiber***
        # Choose from 9 colors in order (TODO: Can add more anticipating more than 9 fibers or bboxes in an image)
        color_list = [(255,0,0),(0,255,0),(0,0,255),(0, 128, 255),(255,255,0),(0,255,255),(128,0,255),(255,0,255),(0,0,0)]
        for current_color,item in enumerate(longest_path_curves):
            cv2.polylines(im, [np.int32(item)], False,color_list[current_color],3)
        
        # Save processed image
        save_path = os.path.join(combined_results_path,d)
        cv2.imwrite(save_path, im)
                

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

...

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
C:\Users\ykocoglu\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

In [70]:
cv2.imshow('test',new_frame*255)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [55]:
longest_path = longest_shortest_path(new_frame,tolerance = 20,min_angle = np.pi/3, turn_on_rdp = False)

In [100]:
len(longest_path)

1

In [91]:
# Get skeleton of given binary image
turn_on_rdp = False

skel = medial_axis(new_frame).astype(np.uint8)

# build graph from the skeleton
graph = sknw.build_sknw(skel)

# create_node_pairs from endpoint nodes:
end_point_nodes = get_end_point_nodes(graph,skel)
new_pts = np.array([graph.nodes[i]['o'] for i in end_point_nodes])
all_pts = np.array([graph.nodes[i]['o'] for i in graph.nodes])
paired_end_points = list(itertools.combinations(end_point_nodes,2))

if len(paired_end_points) == 0:
    print('Not enough endpoints found')
    longest_path = 0

else:
    # Find longest path
    all_paths = []
    for s,e in paired_end_points:
        shortest_path = nx.shortest_path(graph,s,e)
        edge_pairs = list(sliding_window(shortest_path,2))
        sorted_edge_pairs = [tuple(sorted(item)) for item in edge_pairs]


        edge_path = []
        for ss,ee in sorted_edge_pairs:
            edge_segment = [x for x in graph[ss][ee]['pts']]
            edge_path.extend(edge_segment)

        all_paths.append(edge_path)



    longest_path_index = [len(x) for x in all_paths].index(max([len(x) for x in all_paths]))
    longest_path = np.array(all_paths[longest_path_index])

    # If rdp algorithm is turned on, use the rdp algorithm to find the longest path (By default, it's turned off)
    if turn_on_rdp:
        print('Applying rdp')
        # check for turning points with significant angles (based on given tolerance and mind_angle threshold)
        # If turning points exist find another path where no significant turning points exist
        idx = find_turning_points(longest_path,tolerance = 20,min_angle = np.pi/3)

        while len(idx) != 0:
            all_paths.pop(longest_path_index)
            longest_path_index = [len(x) for x in all_paths].index(max([len(x) for x in all_paths]))
            longest_path = np.array(all_paths[longest_path_index])
            idx = find_turning_points(longest_path,tolerance = 20,min_angle = np.pi/3)

In [92]:
longest_path

array([[ 394, 1143],
       [ 394, 1135],
       [ 394, 1129],
       ...,
       [ 769,  370],
       [ 770,  369],
       [ 771,  368]], dtype=int16)

In [69]:
np.unique(new_frame)

array([0, 1], dtype=uint8)